In [1]:
import numpy as np
import pandas as pd
from gurobipy import *

In [5]:
code_dict = {24643: 'Alcoa',
             59176: 'Amex',
             19561: 'Boeing',
             14541: 'Chev',
             11308: 'Coke',
             11703: 'Du Pont',
             22592: 'MMM',
             18163: 'P&G',
             14322: 'Sears',
             17830: 'U Tech'}

In [6]:
mean = np.array([.015617, .019477, .01907, .015801, .021643, .016010, .014892, .016248,.014075, .014537])
std = np.array([.088308, .084585, .01004, .086215, .059886, .068767, .058162, .056385, .080047, .082125])
corr = np.array([[ 1,  0.366, 0.3457,   0.1606,     0.2279,  0.5133,  0.5203,    0.2176,  0.3267,  0.5101], 
             [ 0.366,  1,  0.5379,    0.2165,  0.4986,   0.5823,  0.5569,    0.4760,  0.6517,  0.5853],
             [0.3457,  0.5379,   1,     0.2218,  0.4283,  0.4051,  0.4492,   0.3867,  0.4883,  0.6569],
             [0.1606,  0.2165,   0.2218, 1,    0.0569,  0.3609,  0.2325,    0.2289,  0.1726,  0.3814 ],
             [0.2279,  0.4986,   0.4283, 0.0569,  1,     0.3619,  0.4811,    0.5952,  0.4378,  0.4368],
             [0.5133,  0.5823,   0.4051, 0.3609,  0.3619,  1,     0.6167,    0.4996,  0.5811,  0.5644],
             [0.5203,  0.5569,   0.4492, 0.2325,  0.4811,  0.6167,  1,      0.6037,  0.5671,  0.6032 ],
             [0.2176,  0.4760,   0.3867, 0.2289,  0.5952,  0.4996,  0.6037,      1,  0.5012,  0.4772 ],
             [0.3267,  0.6517,   0.4883, 0.1726,  0.4378,  0.5811,  0.5671,    0.5012,  1,    0.6039 ],
             [ 0.5101, 0.5853,   0.6569, 0.3814,  0.4368,  0.5644,  0.6032,    0.4772,  0.6039,   1 ]])
cov = np.zeros((10, 10))
for i in range(10):
    for j in range(10):
        cov[i][j] = corr[i][j]*std[i]*std[j]

In [7]:
#Build basic portfolio
t = 50
a = 2/t
N = len(mean)
stocks = list(code_dict.values())

m = Model("Basic Portfolio")
w = pd.Series(m.addVars(stocks, lb = 0, ub = GRB.INFINITY), index=stocks)
m.update()

#Set Objective Function
obj = mean.dot(w) - 0.5*a*cov.dot(w).dot(w)
m.setObjective(obj,GRB.MAXIMIZE)
m.addConstr(w.sum() == 1, 'budget')
m.update()

m.setParam('OutputFlag', 0)
m.optimize()

CE_0 = m.ObjVal

Academic license - for non-commercial use only


In [8]:
[w[i].x for i in range(len(w))]

[3.067978726294023e-09,
 5.5127888501443564e-08,
 2.9975867538289806e-08,
 6.886942832482776e-09,
 0.999999871194303,
 7.778843264663669e-09,
 6.522277785280099e-09,
 7.490807305599044e-09,
 5.58240296158643e-09,
 6.372687858321544e-09]

In [10]:
#Parameter Setting
k_list = np.arange(0.05, 0.20, 0.05)
iter_num = 100

In [11]:
#Examine the effect of errors in means
for k in k_list:
    CEL = 0
    for n in range(iter_num):
        z = np.random.normal(size = N)*k + 1
        mean_new = np.multiply(mean, z)
    
        m = Model("Mean Sensitivity")
        w = pd.Series(m.addVars(stocks, lb = 0, ub = GRB.INFINITY), index=stocks)
        m.update()

        #Set Objective Function
        obj = mean_new.dot(w) - 0.5*a*cov.dot(w).dot(w)
        m.setObjective(obj,GRB.MAXIMIZE)
        m.addConstr(w.sum() == 1, 'budget')
        m.update()
        
        m.setParam('OutputFlag', 0)
        m.optimize()
        
        w = np.array([w[i].x for i in range(len(w))])
        CE_x = np.dot(mean,w)- 0.5*a*np.dot(np.dot(w.T, cov), w)
        CEL += (1 - CE_x/CE_0)
    
    CEL /= iter_num
    print(CEL)

0.013225309661817846
0.04262492041165247
0.07223904719047163
0.07989445170315967


In [12]:
#Examine the effect of errors in variances
for k in k_list:
    CEL = 0
    n = 0
    while n < iter_num:
        try:
            z = np.random.normal(size = N)*k + 1
            var = np.diag(cov)
            var_new = np.multiply(var, z)
            cov_new = cov + np.diag(var_new - var)
    
            m = Model("Variance Sensitivity")
            w = pd.Series(m.addVars(stocks, lb = 0, ub = GRB.INFINITY), index=stocks)
            m.update()

            #Set Objective Function
            obj = mean.dot(w) - 0.5*a*cov_new.dot(w).dot(w)
            m.setObjective(obj,GRB.MAXIMIZE)
            m.addConstr(w.sum() == 1, 'budget')
            m.update()
        
            m.setParam('OutputFlag', 0)
            m.optimize()
        
            w = np.array([w[i].x for i in range(len(w))])
            CE_x = np.dot(mean,w)- 0.5*a*np.dot(np.dot(w.T, cov), w)
            CEL += (1 - CE_x/CE_0)
            n += 1
        except GurobiError:
            continue
    
    CEL /= iter_num
    print(CEL)

2.554113049724194e-08
5.549432761653428e-08
6.453852478327527e-08
9.893651162218298e-08


In [13]:
#Examine the effect of errors in variances
for k in k_list:
    CEL = 0
    n = 0
    while n < iter_num:
        try:
            z = np.random.normal(size = N**2)*k + 1
            var = np.diag(cov)
            upper = np.triu(cov)
            cov_new = np.multiply(upper.reshape(-1), z).reshape((N,N))
            cov_new += cov_new.T
            cov_new = cov_new - np.diag(np.diag(cov_new)) + np.diag(var)
    
            m = Model("Variance Sensitivity")
            w = pd.Series(m.addVars(stocks, lb = 0, ub = GRB.INFINITY), index=stocks)
            m.update()

            #Set Objective Function
            obj = mean.dot(w) - 0.5*a*cov_new.dot(w).dot(w)
            m.setObjective(obj,GRB.MAXIMIZE)
            m.addConstr(w.sum() == 1, 'budget')
            m.update()
        
            m.setParam('OutputFlag', 0)
            m.optimize()
        
            w = np.array([w[i].x for i in range(len(w))])
            CE_x = np.dot(mean,w)- 0.5*a*np.dot(np.dot(w.T, cov), w)
            CEL += (1 - CE_x/CE_0)
            n += 1
        except GurobiError:
            continue
            
    CEL /= iter_num
    print(CEL)

3.0391402294771555e-08
5.454894060452631e-08
6.914569309235396e-08
4.274701154272087e-08
